# The Need for Speed: An Introduction to Cython


<img src="figures/cython_logo.png" style="display:block;margin:auto;width:70%;"/>

# Do you really need the speed?

* Write your Python program.
* Ensure it executes correctly and does what it is supposed to.
* Is it fast enough?
* If yes: Ignore the rest of the presentation.
* If no:
    1. Get it right.
    2. Test it's right.
    3. Profile if slow.
    4. Optimise (C/C++ or use Cython and save yourself the pain).
    5. Repeat from 2.
    
> We *should forget* about small efficiencies, say about 97% of the time: **premature optimization is the root of all evil**.

> Yet we should not pass up our opportunities in that critical 3%. A good programmer will not be lulled into complacency by such reasoning, he will be wise to look carefully at the critical code; but only **after** that code has been identified.

<p style="text-align:right">**Donald Knuth**</p>

# What is Cython?

* Cython is a superset of the Python language.
* Cython translates Python code to equivalent C code.
* This code is executed within the CPython runtime environment, but at the speed of compiled C and with the ability to call directly into C libraries. 
* At the same time, it keeps the original interface of the Python source code, which makes it directly usable from Python code.
* This enables Cython's two major use cases:
    1. Extending the CPython interpreter with fast binary modules.
    2. Interfacing Python code with external C libraries.
* While Cython can compile (most) regular Python code, the generated C code usually gains major (and sometime impressive) speed improvements from optional static type declarations for both Python and C types.

# Why is Python slow?

* Contrary to popular belief, Python is **not** slow because it is interpreted. You could use Cython to convert Python to C and compile it and you would gain minimal speedups.
* The main reason that Python is slow is that it's **dynamically** typed.
* At the time the program executes, the interpreter *doesn't know the type of the variables* that are defined. Everything is an *object*, everything is in a *box* which results in **boxing–unboxing overhead**.
* There is also significant overhead on attribute lookups. For instance, calling a method `f` on an object `a` will cause possible lookups in `__dict__`, calls to `__getattr__`, etc, then finally call `__call__` on the callable object `f`. This process cannot be optimised for the reason metioned above.

<img src="figures/boxing_1.png" style="display:block;margin:auto;width:70%;"/>

<img src="figures/boxing_2.png" style="display:block;margin:auto;width:70%;"/>

# Building Cython code

Cython code must, unlike Python, be compiled. This happens in two stages:
* A `.pyx` file is compiled by Cython to a `.c` file, containing the code of a Python extension module,
* The `.c` file is compiled by a `C` compiler to a `.so` file (or `.pyd` on Windows) which can be imported directly into a Python session.
    
There are several ways to build Cython code:
* Write a distutils (or setuptools) `setup.py`.
* Use `pyximport`, importing Cython `.pyx` files as if they were `.py` files (using distutils to compile and build in the background).
* Run the cython command-line utility manually to produce the `.c` file from the `.pyx` file, then manually compiling the `.c` file into a shared object library or DLL suitable for import from Python. (These manual steps are mostly for debugging and experimentation.)
* Use the [Jupyter notebook](http://cython.readthedocs.io/en/latest/src/quickstart/build.html#jupyter), which allows Cython code inline.

# Cython workflow

<img src="figures/compilation.png" style="display:block;margin:auto;width:100%;"/>

# Cython Hello World

1. Create a `helloworld.pyx`:
```Python
print("Hello World")
```
2. Create a `setup.py`, which is like a Python makefile:
```Python
from distutils.core import setup
from Cython.Build import cythonize
setup(ext_modules = cythonize('helloworld.pyx'))
```
3. Build the Cython file:
```Bash
python setup.py build_ext --inplace
```
4. This will leave a file in your local directory called `helloworld.so` in unix or `helloworld.pyd` in Windows. To use this file, simply import it as if it was a regular python module:
```Python
>>> import helloworld
Hello World
```

# `pyximport`: Cython compilation the easy way

If your module doesn't require any extra C libraries or a special build setup, then you can use the `pyximport` module to load `.pyx` files directly on import, without having to write a `setup.py` file:
```Python
>>> import pyximport; pyximport.install()
>>> import helloworld
Hello World
```
It is also possible to compile new `.py` modules that are being imported (including the standard library and installed packages). For using this feature, just tell that to `pyximport`:
```Python
>>> pyximport.install(pyimport=True)
```

# Cython file types

There are three file types in Cython:
1. **Implementation** files carry a `.pyx` suffix. Can contain basically anything Cythonic.
2. **Definition** files carry a `.pxd` suffix. They work like C header files and contain Cython declarations (and sometimes code sections), which are only meant for inclusion by Cython modules. A `.pxd` file is imported into a `.pyx` module by using the `cimport` keyword. Use `cimport` as you would Python’s import statement, to access these files from other definition or implementation files.
3. **Include** files which carry a `.pxi` suffix. Can contain any Cythonic code, because the entire file is textually embedded at the location you prescribe. Include the `.pxi` file with an `include` statement like: `include "spamstuff.pxi"`
`cimport`


# Variable and type definitions

The `cdef` statement is used to declare C variables, either local or module-level
```Cython
cdef int i, j, k
cdef float f, g[42], *h
```
and C `struct`, `union` or `enum` types:
```Cython
cdef struct Grail:
    int age
    float volume

cdef union Food:
    char *spam
    float *eggs

cdef enum CheeseType:
    cheddar, edam,
    camembert

cdef enum CheeseState:
    hard = 1
    soft = 2
    runny = 3
```
Grouping multiple C declarations:
```Cython
cdef:
    struct Spam:
        int tons

    int i
    float a
    Spam *p

    void f(Spam *s):
        print(s.tons, "Tons of spam")
```

# Functions and Methods

1. `def`: callable from Python.
    * Declared with the `def` statement.
    * Called with Python objects.
    * Return Python objects.  
<br />
2. `cdef`: callable from Cython or C.
    * Declared with the `cdef` statement.
    * Called with either Python objects or C values.
    * Return either Python objects or C values.  
<br />
3. `cpdef`: callable from Python, Cython or C.
    * Declared with the `cpdef` statement.
    * Can be called from anywhere, because it uses a little Cython magic.
    * Uses the faster C calling conventions when being called from other Cython code.

# Arrays, `numpy` and memoryviews

* Python has a builtin array module supporting dynamic 1-dimensional arrays of primitive types. It is possible to access the underlying C array of a Python array from within Cython.
* Before typed memoryviews, Cython had different syntax for working efficiently with `numpy` arrays and other buffer-supporting objects. This original buffer syntax is still in use, but it has been superseded by typed memoryviews, which provide more features and cleaner syntax.

```Cython
cimport numpy as np

def convolve(np.ndarray[double, ndim=2] f, np.ndarray[double, ndim=2] g):
    cdef:
        np.ndarray[double, ndim=2] h
        # ...other static declarations
    h = np.zeros((xmax, ymax), dtype=np.double_t)
```

* Typed memoryviews allow efficient access to memory buffers (such as `numpy` arrays, C arrays, or C++ vectors) *without any Python overhead*. They can work with a wide range of buffer-supporting objects: `numpy` arrays, Python memoryview objects, `array.array` objects, and any other type that supports the new buffer protocol. They can also work with C arrays. They
are therefore more general than the `numpy` array buffer syntax, which is restricted to work with `numpy` arrays only.

```Cython
def convolve(double[:, ::1] f, double[:, ::1] g):
    cdef:
        double[:, ::1] h
        # ...
```

# Typed memoryviews

<img src="figures/memoryviews.png" style="display:block;margin:auto;width:60%;"/>

```Cython
cdef int[:, :, :] mv  # a 3D typed memoryview, can be assigned to...
cdef int a[3][3][3]  # 1: a C-array:
a = np.zeros((10,20,30), dtype=np.int32) # 2: a NumPy-array:
cdef int[:, :, :] a = b  # 3: another memoryview

# indexing like NumPy, but faster, at C-level.
mv[1,2,0]  # -> integer
# slicing like numpy, but faster.
mv[10] == mv[10, :, :] == mv[10,...]  # -> a new memoryview.
```

# Extension types (`cdef` classes)

* Somewhat restricted compared to Python classes, but generally more memory efficient and faster. The main difference is that they use a C struct to store their fields and methods instead of a Python dict.
* This allows them to store arbitrary C types in their fields without requiring a Python wrapper for them, and to access fields and methods directly at the C level without passing through a Python dictionary lookup.
* Normal Python classes can inherit from `cdef` classes, but not the other way around. They can also inherit from any number of Python classes and extension types, both in Cython code and pure Python code.
* We cannot use `cdef` and `cpdef` to define methods on non-`cdef` classes.


```Cython
cdef class Particle: # Creates a new type, like list, int, dict
    cdef float *vel, *pos  # attributes stored in instance’s struct
    cdef public float m  # expose variable to Python.
    
    def __cinit__(self, float m, p, v):  # allocate C-level data,
        self.m = m  # called before __init__()
        self.vel = malloc(3*sizeof(float))
        self.pos = malloc(3*sizeof(float))
        # check if vel or pos are NULL...
        for i in range(3):
            self.vel[i] = v[i]; self.pos[i] = p[i]
            
    cpdef apply_impulse(self, f, t):  # methods can be def, cdef, or cpdef.
        # ...
        
    def __dealloc__(self):  # deallocate C arrays, called when gc’d.
        if self.vel: free(self.vel)
        if self.pos: free(self.pos)
```

# Loops

* Cython supports `for` and `while` loops without modification.
* Loops often occupy the majority of a program's runtime, so we can translate Python looping constructs into efficient C analogues.
* If the index variable `i` and range argument `n` are dynamically typed, Cython may not be able to generate a fast C for loop:

```Cython
    n = 100
    for i in range(n):
        # ...
```

* When looping over a range call, we should type the range argument as a C integer:

```Cython
cdef int n
for i in range(n):
    # ...
```

* Cython will automatically type the loop index variable `i` as an `int` as well, provided we *do not use the index in an expression in the loop body*, because it cannot automatically infer whether the operation will overflow, and refuses to infer a C integer type. If we are certain the expression will not cause integer overflow, we statically type the index variable:

```Cython
cdef int i, n
for i in range(n):
    a[i] = i + 1
```

*  For efficient loops over containers, consider converting the container to a C++ equivalent container or using typed memoryviews.

# Automatic type inference

* Cython infers variable types only when doing so cannot change the semantics of the code.
* When enabling `infer_types`, we are taking responsibility to ensure that integer operations do not overflow and that semantics do not change from the untyped version.

```Cython
def automatic_inference():
    i = 1  # may not be representable as C long
    d = 2.0  # inferred that d can be stored as double
    c = 3 + 4j
    r = i * d + c
    return r

cimport cython

@cython.infer_types(True)
def forced_inference():
    i = 1  # inferred as C long, we are taking responsibility for overflows
    d = 2.0
    c = 3 + 4j
    r = i * d + c
    return r
```

In [1]:
%load_ext cython

In [2]:
%%cython -a
def py_fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

def cy_fib(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

In [3]:
%timeit -n 10000 -r 3 py_fib(1000)
%timeit -n 10000 -r 3 cy_fib(1000)

43.5 µs ± 758 ns per loop (mean ± std. dev. of 3 runs, 10000 loops each)
533 ns ± 10.9 ns per loop (mean ± std. dev. of 3 runs, 10000 loops each)


In [4]:
def py_primes(kmax):
    """Calculation of prime numbers in standard Python syntax."""

    p = [0] * 1000
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

print(py_primes(10))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [5]:
%%cython -a
def cy_primes(int kmax):
    """Calculation of prime numbers in Cython."""
    cdef int n, k, i
    cdef int p[1000]
    result = []
    if kmax > 1000:
        kmax = 10000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

print(cy_primes(10))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [6]:
%timeit -n 10000 -r 3 py_primes(20)
%timeit -n 10000 -r 3 cy_primes(20)

40 µs ± 304 ns per loop (mean ± std. dev. of 3 runs, 10000 loops each)
1.21 µs ± 32.8 ns per loop (mean ± std. dev. of 3 runs, 10000 loops each)


In [7]:
# The Levenshtein distance between two words is the minimum number of single-character edits
# (insertions, deletions or substitutions) required to change one word into the other.

def py_levenshtein(s, t):
    return py_lev(s, len(s), t, len(t))

def py_lev(s, len_s, t, len_t):
    if len_s == 0 or len_t == 0:
        return len_s or len_t
    return min(py_lev(s, len_s-1, t, len_t) + 1,
               py_lev(s, len_s, t, len_t-1) + 1,
               py_lev(s, len_s-1, t, len_t-1) + py_cost(s, len_s, t, len_t)) 

def py_cost(s, len_s, t, len_t):
    return s[len_s-1] != t[len_t-1]

In [8]:
%%cython -a
def cy_levenshtein(s, t):
    return lev(s, len(s), t, len(t))

cdef int lev(char *s, int len_s, char *t, int len_t):
    if len_s == 0 or len_t == 0:
        return len_s or len_t
    cdef:
        int lev_s = lev(s, len_s-1, t, len_t  ) + 1
        int lev_t = lev(s, len_s, t, len_t-1) + 1
        int lev_b = lev(s, len_s-1, t, len_t-1) + cost(s, len_s, t, len_t)
    if lev_s < lev_t and lev_s < lev_b:
        return lev_s
    elif lev_t < lev_s and lev_t < lev_b:
        return lev_t
    else:
        return lev_b

cdef int cost(char *s, int len_s, char *t, int len_t):
    return s[len_s-1] != t[len_t-1]

In [9]:
a = b'abcdefgh'
b = b'adqdfyhi'
%timeit -n 50 -r 3 py_levenshtein(a, b)
%timeit -n 50 -r 3 cy_levenshtein(a, b)

143 ms ± 498 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)
1.28 ms ± 95 µs per loop (mean ± std. dev. of 3 runs, 50 loops each)


# Wrapping C code

**`levenshtein.h`**
```C
#ifndef LEVENSHTEIN_H__
#define LEVENSHTEIN_H__

int levenshtein_dist(const char *s, const char *t);

#endif
```

**`levenshtein.c`**
```C
#include "levenshtein.h"
#include <string.h>
#include <stdlib.h>
#define MIN2(a, b) ((a) < (b) ? (a) : (b))
#define MIN3(a, b, c) (MIN2((c), MIN2((a), (b))))

int levenshtein_dist(const char *s, const char *t) {
    int i=0, j=0;
    int m=strlen(s), n=strlen(t);
    int *d = (int*)malloc((m+1) * (n+1) * sizeof(int));
    if (d == NULL) goto fail;

    for (i=1; i<m+1; ++i)
        d[i*(n+1)] = i;

    for (j=0; j<n+1; ++j)
        d[j] = j;

    for (j=1; j<n+1; ++j) {
        for (i=1; i<m+1; ++i) {

            if (s[i-1] == t[j-1]) {
                d[i*(n+1) + j] = d[(i-1) * (n+1) + (j-1)];
            }
            else {
                d[i*(n+1) + j] = MIN3(d[(i-1) * (n+1) +  j   ] + 1,
                                      d[ i    * (n+1) + (j-1)] + 1,
                                      d[(i-1) * (n+1) + (j-1)] + 1);
            }
        }
    }

    int out = d[m * (n+1) + n];

    if(d)
        free(d);
    return out;

    fail:
        return -1;
}
```

**`levenshtein.pyx`**
```Cython
cdef extern from "levenshtein.h":
    int levenshtein_dist(char *s, char *t)

def levenshtein_distance(s, t):
    ld = levenshtein_dist(s, t)
    if ld < 0:
        raise RuntimeError()
    return ld
```

# References

1. [Official Cython Documentation](http://docs.cython.org/en/latest)
* [Python Wiki - Performance Tips](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)
* [StackOverflow - Why are Python Programs often slower than the Equivalent Program Written in C or C++?](http://stackoverflow.com/questions/3033329/why-are-python-programs-often-slower-than-the-equivalent-program-written-in-c-or)
* [P. Virtanen - The Quest for Speed (intro): Interfacing to C with Cython (Advanced Scientific Programming in Python Summer School, 2011)](https://python.g-node.org/python-summerschool-2011/_media/materials/cython/cython-slides.pdf)
* [K. Smith - Cython: Blend the Best of Python and C++ (SciPy 2015 Tutorial)](https://www.youtube.com/watch?v=gMvkiQ-gOW8)
* K. Smith - Cython, A Guide for Python Programmers (2015), O' Reilly Media